# CADS API Python client Tests

In [ ]:
import cads_api_client

In [ ]:
api_url = "http://localhost:8080"

## Client

The client expose the APIs for both 
- catalogue exploration 
- data retrieve

In [ ]:
client = cads_api_client.ApiClient(f"{api_url}/api")
client

In [ ]:
client.url

## Catalogue Exploration

The client 

In [ ]:
collections = client.collections()
collections.response.url

In [ ]:
# access to request response
collections.response

In [ ]:
# list of available collections
collections.collection_ids()

In [ ]:
# select a collection
collection = client.collection("reanalysis-era5-pressure-levels")
collection

In [ ]:
# access to request response
collection.response

**explore collection information**

In [ ]:
collection.end_datetime()

In [ ]:
collection.json["id"]

In [ ]:
collection.json["description"]

## Data Retrieve

### high level API: retrieve using client API

the **retrieve** function is blocking: 
- submits the request
- waits until the requests is completed
- downloads the data

In [ ]:
output_path = client.retrieve(
    collection_id="reanalysis-era5-pressure-levels",
    target="output.grib", # optional
    product_type="reanalysis", 
    format="grib", 
    variable="temperature", 
    pressure_level="1", 
    year=["1971"], 
    month="01", 
    day="25", 
    time="06:00"
)

output_path

In [ ]:
ls output.grib

**list of all submitted requests**

In [ ]:
requests = client.get_requests()
requests.job_ids()

In [ ]:
requests.response.url

TODO: add access to a single job

### retrieve using collection API

**request submission**

In [ ]:
collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    target="output.grib", # optional
    product_type="reanalysis", 
    format="grib", 
    variable="temperature", 
    pressure_level="1", 
    year=["1971"], 
    month="01", 
    day="25", 
    time="06:00"
)

**request monitoring**

In [ ]:
remote.status

**download** function it blocking: 
- waits until the requests is completed
- downloads the data

In [ ]:
remote.download("output.grib")

## Retrieve advanced usage (TBD)

### explore processes

In [ ]:
processing = client.retrieve_api
processing

**list of available processes**

In [ ]:
processes = processing.processes()
processes.process_ids()

**select a process**

In [ ]:
process = processing.process("reanalysis-era5-pressure-levels")
process

In [ ]:
process.json

### submitting a process

In [ ]:
status_info = process.execute(
    inputs=dict(
        product_type="reanalysis", 
        format="grib", 
        variable="temperature", 
        pressure_level="1", 
        year=["1971"], 
        month="01", 
        day="25", 
        time="06:00"
    )
)
status_info

### monitoring

In [ ]:
remote = statusinfo.make_remote()

In [ ]:
remote.status

**Re-create a remote from the request id**

In [ ]:
request_uid = remote.request_uid
request_uid

In [ ]:
processing.make_remote(request_uid)

### download result

In [ ]:
result = remote.build_result()
result

In [ ]:
result.json

In [ ]:
result.download("output.grib")

### ERROR MANAGMENT

**WRONG URL**

In [ ]:
client = cads_api_client.ApiClient((f"{api_url}/not_found")
client.collections()

**MISSING COLLECTION**

In [ ]:
client = cads_api_client.ApiClient(f"{api_url}/api")
client.collection("missing_collection")

**WRONG REQUEST**

In [ ]:
client = cads_api_client.ApiClient(f"{api_url}/api")
collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    target="output.grib", # optional
    product_type="reanalysis", 
    format="grib", 
    variable="temperature", 
    pressure_level="1", 
    year=["2222"], 
    month="01", 
    day="25", 
    time="06:00"
)

In [ ]:
remote.status

In [ ]:
remote = collection.retrieve( 
    target="output.grib", # optional
    product_type="reanalysis", 
    format="grib", 
    variable="temperature", 
    pressure_level="1", 
    year=["2222"], 
    month="01", 
    day="25", 
    time="06:00"
)